In [22]:
# 1404-05-18, Mohammad Kadkhodaei
# -------------------------------

NUM_FOG_NODES = 7
NUM_EDGE_NODES = 14
NUM_CLOUD_NODES = 1

# Infrastructure parameters
CLOUD_vCPU = float('inf')  # Elastic
CLOUD_MEM = float('inf')   # Elastic
FOG_vCPU = 4
FOG_MEM = 4  # GB
EDGE_vCPU = 1
EDGE_MEM = 1  # GB


class Infrastructure:
    def __init__(self):
    
    
        self.cloud_nodes = [{'remaining_vcpu': CLOUD_vCPU, 'remaining_mem': CLOUD_MEM} for _ in range(NUM_CLOUD_NODES)]
        # print(self.cloud_nodes)
        self.fog_nodes = [{'remaining_vcpu': FOG_vCPU, 'remaining_mem': FOG_MEM} for _ in range(NUM_FOG_NODES)]
        # print(self.fog_nodes)
        self.edge_nodes = [{'remaining_vcpu': EDGE_vCPU, 'remaining_mem': EDGE_MEM} for _ in range(NUM_EDGE_NODES)]
    
    def get_feature_vector(self):
        """Returns the current state of infrastructure resources"""
        cloud_vcpu = [n['remaining_vcpu'] for n in self.cloud_nodes]
        cloud_mem = [n['remaining_mem'] for n in self.cloud_nodes]
        fog_vcpu = [n['remaining_vcpu'] for n in self.fog_nodes]
        fog_mem = [n['remaining_mem'] for n in self.fog_nodes]
        edge_vcpu = [n['remaining_vcpu'] for n in self.edge_nodes]
        edge_mem = [n['remaining_mem'] for n in self.edge_nodes]
        
        return cloud_vcpu + cloud_mem + fog_vcpu + fog_mem + edge_vcpu + edge_mem    

In [23]:
infra = Infrastructure()
print(infra.get_feature_vector())

[inf, inf, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
